In [67]:
pip install pdf2image pytesseract
# pdf2image - converts PDF docs to high res image files that can be more easily scanned 
   # uses a library called Poppler which performs conversions of PDF to images 
#pytesseract can take the image and pull text from it
   # tesseract is an "Optical Character Recognition" (OCR) engine that can process and extract text from images to convert to machine-readable characters 
   # performs OCR on each PDF page image  

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3121322038.py, line 1)

In [ ]:
from pdf2image import convert_from_path
import pytesseract

# reads all of script normally 
def pdf_to_text_with_ocr(file_path, output_text_file):
   text = ""
   try:
      # Convert each page of the PDF to an image
      pages = convert_from_path(file_path, dpi=300)
      for page in pages:
         # Perform OCR on each image page
         text += pytesseract.image_to_string(page) + "\n"
   except Exception as e:
      print(f"Error during OCR: {e}")

   # Write the accumulated text to the specified text file
   with open(output_text_file, "w", encoding="utf-8") as file:
      file.write(text)
   #  return text

file_path = "short-test.pdf"
output_file = "output.txt"
pdf_to_text_with_ocr(file_path, output_file)


In [81]:
from pdf2image import convert_from_path
import pytesseract
import re

characters = ["LEADING PLAYER", "PIPPIN", "CHARLES", "LEWIS", "FASTRADA", "BERTHE", "CATHERINE", "THEO", "BAND OF PLAYERS", "ALL", "SOLDIERS", "HEAD", "WOMEN 1", "MEN 1", "MEN 2", "WOMEN 2"]

def character_lines_and_songs(file_path, output_text_file):
   text = ""
   try:
      # Convert each page of the PDF to an image
      pages = convert_from_path(file_path, dpi=300)
      for page in pages:
         # Perform OCR on each image page
         page_text = pytesseract.image_to_string(page) + "\n"

         # Remove all text in parentheses
         # page_text = re.sub(r'\(.*?\)', '', page_text)

         formatted_text = ""
         last_character_name = None
         character_name = None
         for line in page_text.splitlines():
            # Strip leading/trailing whitespace and check if the line contains text
            line = line.strip()
            if not line: 
               continue # skip empty lines
            
            # if re.match(r'^[A-Z ]+$', line):
            #    print(line, len(line.split()))
            if re.match(r'^[A-Z ]+$', line) and line in characters:
               character_name = line

               if character_name != last_character_name:
                  formatted_text += f"[{character_name}]: "
                  last_character_name = character_name 
            else: 
               if character_name:
                  formatted_text += f"[{character_name}]: {line.()}\n"
               # If it's in uppercase but part of a song, treat it as a song line
               elif line.isupper():
                  formatted_text += f"[SONG]: {line}\n"
               else:
                  # Otherwise, just add the line as it is (if it's spoken by the character)
                  formatted_text += f"{line}\n"

         # formatted_text = re.sub(r'\(.*?\)', '', page_text)
         text += "\n" + formatted_text

   except Exception as e:
      print(f"Error: {e}")

   # Write the accumulated text to the specified text file
   with open(output_text_file, "w", encoding="utf-8") as file:
      file.write(text)
   #  return text

file_path = "short-test.pdf"
output_file = "lines-and-songs.txt"
character_lines_and_songs(file_path, output_file)


SyntaxError: f-string: invalid syntax (3104813112.py, line 38)

Idea: 
- put all text in text file
- after all text is parsed correctly, the reviewer can look through it and see if there are any mistakes
- then, run the next script which converts that text file into slides

## Current Issues
1. Columns of text are not working - when multiple people are supposed to be talking or singing at once

In [83]:
from pdf2image import convert_from_path
import pytesseract
import re

characters = ["LEADING PLAYER", "PIPPIN", "CHARLES", "LEWIS", "FASTRADA", "BERTHE", "CATHERINE", "THEO", "BAND OF PLAYERS", "ALL", "A PLAYER", "PLAYERS", "OTHER PLAYERS", "SOLDIERS", "HEAD"]

def character_lines_and_songs(file_path, output_text_file):
      text = ""

      pages = convert_from_path(file_path, dpi=300)
      for page in pages:
            # Perform OCR on each image page
            page_text = pytesseract.image_to_string(page) + "\n"

            formatted_text = ""
            character_name = None
            accumulated_line = ""

            for line in page_text.splitlines():
                  # if line does not contain text, then do not add to text
                  line = line.strip()
                  # line starting with # may be specific to pippen script
                  if not line or line[0] == '#': 
                        continue

                  # if character line, then line is only character's name and in all caps 
                  if re.match(r'^[A-Z ]+$', line) and line in characters:
                        if accumulated_line:
                              formatted_text += accumulated_line + "\n"
                              accumulated_line = ""  # Reset the accumulator

                        character_name = line
                        formatted_text += f"\n[{character_name}]: "
                  elif line.isupper():
                        if accumulated_line:
                              formatted_text += accumulated_line + "\n"
                              accumulated_line = ""  # Reset the accumulator
                        
                        formatted_text += f"\n{line}"

                  # else if not character line, then normal line so add to text 
                  else: 
                        accumulated_line += " " + line if accumulated_line else line

            # formatted_text = re.sub(r'\(.*?\)', '', formatted_text)
            if accumulated_line:
                  formatted_text += accumulated_line
            
            formatted_text = re.sub(r'\(.*?\)|\|', '', formatted_text)
            text += formatted_text + "\n"

      # write text to file 
      with open(output_text_file, "w", encoding="utf-8") as file:
            file.write(text)


file_path = "short-test.pdf"
output_file = "lines-songs-2.txt"
character_lines_and_songs(file_path, output_file)

# TODO: if multiple characters on one line, then could mean that multiple people talking with multiple columns of text


### Other todo
to be done separately from function above, 
   - turn google doc or text file into google slides with rules